# DE Analysis with EdgeR

After acquiring a table/matrix of read counts per gene, we can begin conducting DE analysis to determine genes which are differentially expressed in different tissue populations. However, before we can get our list of DE genes, we need to process our data for usage with edgeR.

EdgeR documentation can be found [here](https://bioconductor.org/packages/release/bioc/vignettes/edgeR/inst/doc/edgeRUsersGuide.pdf). This resource has information on the different functions used in this tutorial, as well as a more in-depth explanation of how they work.

In [1]:
library(edgeR)

Loading required package: limma



## Downloading Data

First we need some data to analyze. Lets get some data from a [GEO Submission](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE60450). Download the GSE60450_Lactation-GenewiseCounts.txt.gz file, unzip it, then load it into R.

In [9]:
rawcounts = read.table("GSE60450_Lactation-GenewiseCounts.txt",header=TRUE)

In [10]:
head(rawcounts)

,EntrezGeneID,Length,MCL1.DG_BC2CTUACXX_ACTTGA_L002_R1,MCL1.DH_BC2CTUACXX_CAGATC_L002_R1,MCL1.DI_BC2CTUACXX_ACAGTG_L002_R1,MCL1.DJ_BC2CTUACXX_CGATGT_L002_R1,MCL1.DK_BC2CTUACXX_TTAGGC_L002_R1,MCL1.DL_BC2CTUACXX_ATCACG_L002_R1,MCL1.LA_BC2CTUACXX_GATCAG_L001_R1,MCL1.LB_BC2CTUACXX_TGACCA_L001_R1,MCL1.LC_BC2CTUACXX_GCCAAT_L001_R1,MCL1.LD_BC2CTUACXX_GGCTAC_L001_R1,MCL1.LE_BC2CTUACXX_TAGCTT_L001_R1,MCL1.LF_BC2CTUACXX_CTTGTA_L001_R1
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,497097,3634,438,300,65,237,354,287,0,0,0,0,0,0
2,100503874,3259,1,0,1,1,0,4,0,0,0,0,0,0
3,100038431,1634,0,0,0,0,0,0,0,0,0,0,0,0
4,19888,9747,1,1,0,0,0,0,10,3,10,2,0,0
5,20671,3130,106,182,82,105,43,82,16,25,18,8,3,10
6,27395,4203,309,234,337,300,290,270,560,464,489,328,307,342


When analyzing read count data, its important that there are only as many columns as samples. So lets drop the length column and make the geneID column into the table index

In [ ]:
# fill in the code below
countdata = 

### Defining our groups

Before we can do any analysis, we have to differentiate the types of groups from each other in our RNA-seq experiment (treatment vs control or liver vs heart vs brain or....). To do so, create a vector of the different groups in order of the columns of your read count dataset.

So, if you have a table with two heart columns followed by two liver columns, your group vector could be:

`groups = c("hrt","hrt","liv","liv")`

For the GEO data, there are 6 groups, according to this figure. 

![](https://galaxyproject.github.io/training-material/topics/transcriptomics/images/rna-seq-reads-to-counts/mouse_exp.png)


However for simplicity's sake, lets assume there are only two groups, basal and luminal. In that case, our group vector should be:

In [16]:
mgroups = rep(c(0,1), each = 6)

In [17]:
mgroups

[1] 0 0 0 0 0 0 1 1 1 1 1 1

### Creating a DGEList

EdgeR analyzes data in a specific data object called a DGEList. Lets create one with our data.

In [ ]:
d = DGEList(counts=countdata,group=factor(mgroups))

The beauty of DGELists is that renaming the list does not overwrite the original data, just add more elements to it.

## Filtering the Data

Now lets get rid of genes which don't occur very frequently in either group. Let do this in two ways, one using a threshold we define, and one using edgeR's filtering method. 

For the first, only keep genes that have at least 5 reads in at least 4 samples each.

For the second, use the `filterByExpr` function in the edgeR package.

Lets save both of these filtered methods, and we can compare our list of DE genes using both methods at the end.

## Normalizing the Data

Its important to recognize that raw read count data is NOT directly equal to the in-vivo mRNA levels of transcripts in a population. Because of this, it's important to transform the read counts to more accurately reflect the in-vivo levels. By default, edgeR uses the TMM method to do so. You can read more about how TMM works [here](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2010-11-3-r25) and [here](https://www.frontiersin.org/articles/10.3389/fgene.2016.00164/full).

In summary, TMM corrects for:

* gene length (the longer the gene, the more reads it will have. Not very relevant for DE analysis)
* library size (the more total reads in a sample, the more reads it will have per transcript)
* transcriptome size (the more total DE genes in a sample, the less reads will be registered per other transcripts; think of reading power as constant, and having to be divided between number of transcripts in a cel)



Lets normalize both of our filtered datasets using `calcNormFactors`

In [ ]:
# Fill in the code below!
y_thresh = calcNormFactors()
y_edger = calcNormFactors()

In [ ]:
# What do these lists look like?
y_edger

note that the raw values are not changed. Instead, normalization factors are saved, which will be used later in the calculation

### EXTRA: Why not use other normalization methods?

In addition to algorithms which correct for all of the above mentioned differences in samples/genes, some are much simpler.

* cpm, or counts per million, corrects only for library size
* RPKM/FPKM, or reads/fragments per kilobase per million, corrects for library size and gene size

Implementations for both of these can be found [here](https://sites.google.com/site/wiki4metagenomics/pdf/definition/rpkm-calculation).

Lets do a thought experiment on why we shouldn't use these values in DE analysis. 

Assume you have a sample A with 50 million total reads. How many raw counts would a 40 kb transcript need to have an RPKM value equal to a 10 kb transcript? 

Even though these raw counts are very different, they still have the same RPKM value, making it unideal for comparing differential expression.

## Visualizing the Data

Now that we have normalized values, lets check our data to see how different our groups are from each other. We can do this with an MDS (multidimensional scaling) plot. It works via placing similar samples close to each other, and dissimilar samples far apart. Use the `plotMDS` function to generate the graph.

Note: this is essentially the same as a PCA of the data.

In [ ]:
plotMDS()

How does the plot look? Does it appear that samples of the same group are clustered together? How do the different filtering methods look when their plots are compared to each other?

## Dispersion Estimation

Recognizing how variable samples are from each other within a group is necessary for edgeR to properly calculate a list of DE genes. Use the `estimateDisp` function to estimate dispersions for your two DGELists.

### How does it work?

It's difficult to properly measure dispersion between samples within a group when usually there are very few samples in a group for RNA-sequencing. To account for this issue, edgeR makes some assumptions about the distriution of gene read variances and uses genes with similar values and variances to fill in the necessary data to properly estimate dispersion (common dispersion). And to account for any broad assumptions made from calculating common dispersions, tagwise dispersions are also calculated using a [bayesian approach](https://academic.oup.com/bioinformatics/article/23/21/2881/372869).

## DE Analysis

Now that all our pre-reqs are finished, we can finally conduct DE Analysis. EdgeR uses a glm to generalize model creation for identifying DE genes. It also uses an exact test under specific conditions. Which one should we use in this case? Use it and create a list of DE genes for both DGELists

How do these lists differ? Does the FDR differ between them for each gene? Why or why not? 

## Heatmap Generation

Congrats, we now have a list of DE genes between our groups! However, its always nice to have a visual representation of the differences in transcript levels between samples and between groups. To do this, we can generate a heatmap to indicate differences between samples. 

![](https://i.imgur.com/5DJt2wl.png)

In this example, each row is a gene and each column is a sample. The data in each row is standardized, so a negative value indicates a value below the "average reads" of that row. We can see a strict separation of gene expressions between the two groups.

Now, lets try making our own heatmaps. Create a list of the most differentially expressed genes you identified and create a heatmap. There are a variety of R packages to create heatmaps, including [pheatmap](https://davetang.org/muse/2018/05/15/making-a-heatmap-in-r-with-the-pheatmap-package/) and [heatmap2](https://davetang.org/muse/2010/12/06/making-a-heatmap-with-r/). Heatmaps can also be generated in Python.

Remember to use **normalized** read counts in your heatmap (crucial whenever you are comparing samples) and to standardize your values. Though there are a variety of methods to standardize data, I recommend using z-scores.

How does your heatmap look? Do the two groups separate nicely based on the DE genes? How does the heatmap look if you use another normalization method (cpm or rpkm or tmm)?

## References

The following resources were used to help construct this tutorial:

https://galaxyproject.github.io/training-material/topics/transcriptomics/tutorials/rna-seq-counts-to-genes/tutorial.html

https://www.youtube.com/watch?v=5tGCBW3_0IA

https://web.stanford.edu/class/bios221/labs/rnaseq/lab_4_rnaseq.html

http://78.128.216.61:8081/manual/ngs-dea-edger-RNA.html

https://bioconductor.org/packages/release/bioc/vignettes/edgeR/inst/doc/edgeRUsersGuide.pdf